# Création de l'architecture d'une application PowerApps
## Présentation
Ce script sert à créer l'architecture d'une application PowerApps en partant du fichier DataSource.json de son export de la plateforme.

Le fichier DataSource.json est dans le fichier "application.maspp" dans le répertoire \Microsoft.PowerApps\apps\'numéro de l'application'\.

Pour l'extraire, il faut renommer le fichier "application"-document.msapp en "application"-docment.zip", iIl est situé dans le répertoire : References de ce fichier


In [5]:
import json
import re
from collections import defaultdict

# Charger le fichier DataSources.json
input_file = "./data/anonymized_DataSource.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Accéder à la liste des DataSources
data_sources = data["DataSources"]

# Tables à exclure
excluded_tables = ["Divisions", "Utilisateurs"]

# Colonnes systèmes à exclure
excluded_columns = {
    "utcconversiontimezonecode", "_ownerid_value", "statuscode", "statecode",
    "modifiedonbehalfby", "createdonbehalfby", "owninguser", "timezoneruleversionnumber",
    "owningbusinessunit", "modifiedon", "modifiedby", "versionnumber",
    "overriddencreatedon", "createdby", "importsequencenumber", "owningteam", "createdon"
}

# Dictionnaire pour stocker les tables et leurs colonnes
tables = {}

# Dictionnaire pour les Option Sets
option_sets = {}
option_sets_by_column = {}  # Pour lier rapidement colonne -> choix
option_set_references = {}   # Pour les liaisons entre tables

# Extraire d'abord tous les Option Sets avec leurs références
for ds in data_sources:
    if ds.get("Type") == "OptionSetInfo":
        option_set_name = ds.get("Name")
        related_entity = ds.get("RelatedEntityName")
        related_column = ds.get("RelatedColumnInvariantName")
        option_mapping = ds.get("OptionSetInfoNameMapping", {})
        option_references = ds.get("OptionSetReference", {})
        
        if option_set_name and related_entity and related_column:
            key = f"{related_entity}.{related_column}"
            option_sets_by_column[key] = option_mapping
            
            # Extraire les références vers d'autres tables
            references = []
            if option_references:
                for ref_key, ref_value in option_references.items():
                    if isinstance(ref_value, dict):
                        ref_entity = ref_value.get("OptionSetReferenceEntityName")
                        ref_column = ref_value.get("OptionSetReferenceColumnName")
                        if ref_entity and ref_column:
                            references.append({
                                "entity": ref_entity,
                                "column": ref_column
                            })
            
            if references:
                option_set_references[key] = references

# Parcourir les DataSources pour les tables
for ds in data_sources:
    if ds.get("Type") == "NativeCDSDataSourceInfo":
        table_name = ds.get("Name") or ds.get("LogicalName") or ds.get("EntitySetName")
        if not table_name or table_name in excluded_tables:
            continue
            
        columns_detail = {}
        name_mapping = ds.get("NativeCDSDataSourceInfoNameMapping", {})
        
        if name_mapping:
            for technical_name, display_name in name_mapping.items():
                if technical_name in excluded_columns:
                    continue
                columns_detail[technical_name] = display_name
        
        tables[table_name] = columns_detail

# Fonction pour générer l'export HTML
def generate_html_export(tables, option_sets_by_column, option_set_references):
    html_lines = [
        '<!DOCTYPE html>',
        '<html lang="fr">',
        '<head>',
        '<meta charset="UTF-8">',
        '<meta name="viewport" content="width=device-width, initial-scale=1.0">',
        '<title>Architecture Power Apps</title>',
        '<style>',
        'body { font-family: Arial, sans-serif; margin: 20px; }',
        'h1 { color: #2c3e50; }',
        'h2 { color: #34495e; border-bottom: 2px solid #3498db; padding-bottom: 5px; }',
        'h3 { color: #4a4a4a; }',
        '.table-container { margin-bottom: 30px; }',
        '.columns-table { border-collapse: collapse; width: 100%; margin: 10px 0; }',
        '.columns-table th, .columns-table td { border: 1px solid #ddd; padding: 8px; text-align: left; }',
        '.columns-table th { background-color: #f2f2f2; }',
        '.choices-table { border-collapse: collapse; width: 80%; margin: 10px 0 20px 20px; }',
        '.choices-table th, .choices-table td { border: 1px solid #eee; padding: 6px; text-align: left; }',
        '.choices-table th { background-color: #e8f4f8; }',
        '.choice-link { color: #2980b9; text-decoration: none; }',
        '.choice-link:hover { text-decoration: underline; }',
        '.references { margin: 10px 0; padding: 10px; background-color: #fff3cd; border-left: 4px solid #ffc107; }',
        '.connection-line { border-left: 2px dashed #95a5a6; margin-left: 20px; padding-left: 15px; }',
        '</style>',
        '</head>',
        '<body>',
        '<h1>Architecture Power Apps</h1>'
    ]
    
    # Pour chaque table
    for table, cols in sorted(tables.items()):
        html_lines.append(f'<div class="table-container">')
        html_lines.append(f'<h2>Table: {table} ({len(cols)} colonnes)</h2>')
        
        # Tableau des colonnes
        html_lines.append('<table class="columns-table">')
        html_lines.append('<thead>')
        html_lines.append('<tr><th>Nom Technique</th><th>Nom Affiché</th><th>Liste de Choix</th><th>Références</th></tr>')
        html_lines.append('</thead>')
        html_lines.append('<tbody>')
        
        for tech_name, display_name in sorted(cols.items()):
            option_key = f"{table}.{tech_name}"
            
            # Liste de choix
            choices_html = ""
            if option_key in option_sets_by_column:
                options = option_sets_by_column[option_key]
                choices_html = f'<a href="#choice_{table}_{tech_name}" class="choice-link">Voir {len(options)} choix</a>'
            
            # Références
            references_html = ""
            if option_key in option_set_references:
                refs = option_set_references[option_key]
                ref_links = []
                for ref in refs:
                    ref_links.append(f'{ref["entity"]}.{ref["column"]}')
                references_html = "<br>".join(ref_links)
            
            html_lines.append(f'<tr>')
            html_lines.append(f'<td>{tech_name}</td>')
            html_lines.append(f'<td>{display_name}</td>')
            html_lines.append(f'<td>{choices_html}</td>')
            html_lines.append(f'<td>{references_html}</td>')
            html_lines.append(f'</tr>')
        
        html_lines.append('</tbody>')
        html_lines.append('</table>')
        
        # Tableaux des listes de choix
        for tech_name, display_name in sorted(cols.items()):
            option_key = f"{table}.{tech_name}"
            if option_key in option_sets_by_column:
                options = option_sets_by_column[option_key]
                html_lines.append(f'<div id="choice_{table}_{tech_name}" class="connection-line">')
                html_lines.append(f'<h3>Liste de choix pour {display_name} ({tech_name})</h3>')
                html_lines.append('<table class="choices-table">')
                html_lines.append('<thead><tr><th>Code</th><th>Valeur</th></tr></thead>')
                html_lines.append('<tbody>')
                
                for key, value in sorted(options.items()):
                    html_lines.append(f'<tr><td>{key}</td><td>{value}</td></tr>')
                
                html_lines.append('</tbody>')
                html_lines.append('</table>')
                html_lines.append('</div>')
        
        html_lines.append('</div>')
    
    # Section des références croisées
    html_lines.append('<h2>Références Croisées</h2>')
    html_lines.append('<div class="references">')
    
    if option_set_references:
        for option_key, references in option_set_references.items():
            table_name, col_name = option_key.split('.', 1)
            html_lines.append(f'<p><strong>{table_name}.{col_name}</strong> est référencé par:')
            for ref in references:
                html_lines.append(f'<br>&nbsp;&nbsp;→ {ref["entity"]}.{ref["column"]}')
    else:
        html_lines.append('<p>Aucune référence croisée trouvée.</p>')
    
    html_lines.append('</div>')
    
    html_lines.append('</body>')
    html_lines.append('</html>')
    
    output_file="./data/architecture_powerapps.html"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(html_lines))
    
    print("✅ Fichier 'architecture_powerapps.html' généré")

# Fonction pour nettoyer les noms pour Mermaid
def clean_mermaid_name(name):
    # Remplacer les caractères problématiques
    cleaned = re.sub(r'[^a-zA-Z0-9_]', '_', name)
    # S'assurer qu'il ne commence pas par un chiffre
    if cleaned and cleaned[0].isdigit():
        cleaned = "attr_" + cleaned
    return cleaned if cleaned else "unnamed_attr"

# Fonction pour générer le diagramme Mermaid avec la bonne syntaxe
def generate_mermaid_graph(tables):
    mermaid_lines = ["```mermaid", "erDiagram"]
    
    # Ajouter les entités avec la syntaxe correcte
    for table, cols in tables.items():
        clean_table = clean_mermaid_name(table)
        
        mermaid_lines.append(f'    {clean_table} {{')
        
        # Ajouter les attributs (limité à 10 pour la lisibilité)
        for i, (tech_name, display_name) in enumerate(list(cols.items())[:10]):
            clean_attr = clean_mermaid_name(tech_name)
            # Syntaxe correcte : type nom
            mermaid_lines.append(f'        string {clean_attr} "{display_name}"')
        
        if len(cols) > 10:
            mermaid_lines.append(f'        string other "... et {len(cols) - 10} autres colonnes"')
            
        mermaid_lines.append('    }')
    
    # Ajouter quelques relations basiques
    for table, cols in tables.items():
        clean_table = clean_mermaid_name(table)
        for col_name in cols.keys():
            if col_name.endswith('id') and col_name != 'id':
                target_table = col_name[:-2]  # Enlever 'id'
                if target_table in tables:
                    clean_target = clean_mermaid_name(target_table)
                    clean_col = clean_mermaid_name(col_name)
                    mermaid_lines.append(f'    {clean_target} ||--o{{ {clean_table} : "{col_name}"')
    
    mermaid_lines.append("```")
    return "\n".join(mermaid_lines)

# Export vers format JSON structuré
def generate_structured_json():
    architecture = {
        "tables": {},
        "relationships": [],
        "option_sets": {},
        "option_set_references": option_set_references
    }
    
    # Tables et colonnes
    for table, cols in tables.items():
        architecture["tables"][table] = {
            "columns": cols,
            "option_sets": {}
        }
        
        # Ajouter les option sets pour cette table
        for col_name in cols.keys():
            option_key = f"{table}.{col_name}"
            if option_key in option_sets_by_column:
                architecture["tables"][table]["option_sets"][col_name] = option_sets_by_column[option_key]
    
    # Relations basiques
    for table, cols in tables.items():
        for col_name in cols.keys():
            if col_name.endswith('id') and col_name != 'id':
                target_table = col_name[:-2]
                if target_table in tables:
                    architecture["relationships"].append({
                        "from_table": target_table,
                        "to_table": table,
                        "column": col_name,
                        "type": "lookup"
                    })
    
    return architecture

# Export pour Drawio
def clean_xml_text(text):
    """Nettoyer le texte pour XML"""
    if not text:
        return ""
    return text.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;").replace('"', "&quot;")

def generate_drawio_complete(tables, option_sets_by_column, option_set_references):
    xml_lines = [
        '<?xml version="1.0" encoding="UTF-8"?>',
        '<mxfile host="app.diagrams.net">',
        '<diagram name="Architecture Power Apps" id="architecture-powerapps">',
        '<mxGraphModel dx="1426" dy="745" grid="1" gridSize="10" guides="1" tooltips="1" connect="1" arrows="1" fold="1" page="1" pageScale="1" pageWidth="850" pageHeight="1100" background="#ffffff" math="0" shadow="0">'
    ]
    
    xml_lines.append('<root>')
    xml_lines.append('<mxCell id="0"/>')
    xml_lines.append('<mxCell id="1" parent="0"/>')
    
    # Générer les tables avec toutes les colonnes
    x_pos = 20
    y_pos = 20
    table_positions = {}  # Pour les relations
    used_positions = set()  # Pour éviter les superpositions
    
    # Premier passage : Tables principales avec positionnement intelligent
    for i, (table, cols) in enumerate(tables.items()):
        # Calculer la position avec espacement suffisant
        table_width = 320
        table_height = 30 + len(cols) * 26
        
        # Position en grille avec espacement suffisant
        grid_x = i % 3
        grid_y = i // 3
        spacing_x = 400  # Espacement horizontal
        spacing_y = max(table_height + 100, 300)  # Espacement vertical basé sur la hauteur
        
        table_x = x_pos + grid_x * spacing_x
        table_y = y_pos + grid_y * spacing_y
        
        table_id = f"table_{i}"
        table_positions[table] = {
            'id': table_id,
            'x': table_x,
            'y': table_y,
            'width': table_width,
            'height': table_height
        }
        
        # Table swimlane
        xml_lines.append(f'<mxCell id="{table_id}" value="{clean_xml_text(table)}" style="swimlane;fontStyle=1;align=center;verticalAlign=top;childLayout=stackLayout;horizontal=1;startSize=30;horizontalStack=0;resizeParent=1;resizeParentMax=0;resizeLast=0;collapsible=1;marginBottom=0;swimlaneFillColor=#ffffff;fillColor=#e3f2fd;strokeColor=#90A4AE;" vertex="1" parent="1">')
        xml_lines.append(f'<mxGeometry x="{table_x}" y="{table_y}" width="{table_width}" height="{table_height}" as="geometry"/>')
        xml_lines.append('</mxCell>')
        
        # Ajouter les colonnes (toutes)
        current_y = 30
        for j, (tech_name, display_name) in enumerate(cols.items()):
            attr_id = f"{table_id}_attr_{j}"
            display_text = display_name if display_name else tech_name
            
            # Vérifier s'il y a une liste de choix
            option_key = f"{table}.{tech_name}"
            if option_key in option_sets_by_column:
                # Ajouter un indicateur visuel
                display_text = f"📋 {display_name}"
            
            # Style différent pour les colonnes avec listes de choix
            style = "text;strokeColor=none;fillColor=none;align=left;verticalAlign=middle;spacingLeft=4;spacingRight=4;overflow=hidden;rotatable=0;points=[[0,0.5],[1,0.5]];portConstraint=eastwest;"
            if option_key in option_sets_by_column:
                style = "text;strokeColor=none;fillColor=#fff3e0;align=left;verticalAlign=middle;spacingLeft=4;spacingRight=4;overflow=hidden;rotatable=0;points=[[0,0.5],[1,0.5]];portConstraint=eastwest;"
            
            xml_lines.append(f'<mxCell id="{attr_id}" value="{clean_xml_text(display_text)}" style="{style}" vertex="1" parent="{table_id}">')
            xml_lines.append(f'<mxGeometry y="{current_y}" width="{table_width}" height="26" as="geometry"/>')
            xml_lines.append('</mxCell>')
            
            current_y += 26
    
    # Deuxième passage : Tables de choix avec positionnement évitant les superpositions
    choice_table_id = 1000
    choice_positions = {}
    
    # Pour chaque table, calculer la position des tables de choix à droite
    for table, cols in tables.items():
        table_info = table_positions.get(table)
        if not table_info:
            continue
            
        table_right_edge = table_info['x'] + table_info['width']
        choice_y_start = table_info['y']
        choice_vertical_spacing = 250  # Espacement vertical entre les tables de choix
        choice_count = 0
        
        for tech_name, display_name in cols.items():
            option_key = f"{table}.{tech_name}"
            if option_key in option_sets_by_column:
                options = option_sets_by_column[option_key]
                
                # Créer une table de choix avec positionnement non superposé
                choice_id = f"choice_{choice_table_id}"
                choice_table_id += 1
                
                # Position à droite de la table principale, avec décalage vertical
                choice_x = table_right_edge + 50
                choice_y = choice_y_start + (choice_count * choice_vertical_spacing)
                choice_count += 1
                
                # Vérifier si la position est déjà utilisée et ajuster si nécessaire
                while (choice_x, choice_y) in used_positions:
                    choice_y += 50  # Décaler légèrement
                
                used_positions.add((choice_x, choice_y))
                
                choice_positions[option_key] = {
                    'id': choice_id,
                    'x': choice_x,
                    'y': choice_y
                }
                
                # Hauteur selon le nombre d'options
                choice_height = 30 + len(options) * 20
                
                # Table de choix
                xml_lines.append(f'<mxCell id="{choice_id}" value="{clean_xml_text(display_name)} [Choix]" style="swimlane;fontStyle=1;align=center;verticalAlign=top;childLayout=stackLayout;horizontal=1;startSize=30;horizontalStack=0;resizeParent=1;resizeParentMax=0;resizeLast=0;collapsible=1;marginBottom=0;swimlaneFillColor=#ffffff;fillColor=#f3e5f5;strokeColor=#AB47BC;" vertex="1" parent="1">')
                xml_lines.append(f'<mxGeometry x="{choice_x}" y="{choice_y}" width="240" height="{choice_height}" as="geometry"/>')
                xml_lines.append('</mxCell>')
                
                # Ajouter les options (toutes) triées par ordre croissant
                current_y = 30
                sorted_options = sorted(options.items(), key=lambda x: x[0])
                for k, (option_key_val, option_value) in enumerate(sorted_options):
                    option_attr_id = f"{choice_id}_option_{k}"
                    option_display_text = f"{option_key_val} → {option_value}"
                    xml_lines.append(f'<mxCell id="{option_attr_id}" value="{clean_xml_text(option_display_text)}" style="text;strokeColor=none;fillColor=none;align=left;verticalAlign=middle;spacingLeft=4;spacingRight=4;overflow=hidden;rotatable=0;points=[[0,0.5],[1,0.5]];portConstraint=eastwest;fontSize=9;" vertex="1" parent="{choice_id}">')
                    xml_lines.append(f'<mxGeometry y="{current_y}" width="240" height="20" as="geometry"/>')
                    xml_lines.append('</mxCell>')
                    current_y += 20
    
    # Troisième passage : Relations avec chemins évitant les superpositions
    
    # 1. Relations entre colonnes et tables de choix
    edge_id = 5000
    for table, cols in tables.items():
        table_info = table_positions.get(table)
        if not table_info:
            continue
            
        for tech_name, display_name in cols.items():
            option_key = f"{table}.{tech_name}"
            if option_key in option_sets_by_column and option_key in choice_positions:
                # Trouver l'ID de la colonne
                table_id = table_info['id']
                try:
                    col_index = list(cols.keys()).index(tech_name)
                    attr_id = f"{table_id}_attr_{col_index}"
                    
                    choice_info = choice_positions[option_key]
                    
                    # Créer la relation avec style évitant les superpositions
                    source_cell = attr_id
                    target_cell = choice_info['id']
                    xml_lines.append(f'<mxCell id="edge_{edge_id}" style="edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;entryX=0;entryY=0.5;startArrow=none;endArrow=block;endFill=1;strokeColor=#666666;strokeWidth=1;fontSize=9;" edge="1" parent="1" source="{source_cell}" target="{target_cell}">')
                    xml_lines.append('<mxGeometry relative="1" as="geometry"/>')
                    xml_lines.append('</mxCell>')
                    edge_id += 1
                except (ValueError, IndexError):
                    continue
    
    # 2. Relations entre tables (colonnes se terminant par 'id')
    for table, cols in tables.items():
        table_info = table_positions.get(table)
        if not table_info:
            continue
        table_id = table_info['id']
        
        for col_name in cols.keys():
            if col_name.endswith('id') and col_name != 'id':
                target_table = col_name[:-2]  # Enlever 'id'
                if target_table in table_positions:
                    target_info = table_positions[target_table]
                    target_id = target_info['id']
                    
                    # Trouver l'ID de la colonne source
                    try:
                        col_index = list(cols.keys()).index(col_name)
                        source_attr_id = f"{table_id}_attr_{col_index}"
                        
                        # Créer la relation avec style fléché
                        source_cell = source_attr_id
                        target_cell = target_id
                        edge_label = clean_xml_text(col_name)
                        xml_lines.append(f'<mxCell id="edge_{edge_id}" value="{edge_label}" style="edgeStyle=elbowEdgeStyle;html=1;endArrow=ERoneToMany;entryX=0;entryY=0.5;strokeColor=#2196F3;strokeWidth=2;fontSize=10;elbow=vertical;" edge="1" parent="1" source="{source_cell}" target="{target_cell}">')
                        xml_lines.append('<mxGeometry width="100" height="100" relative="1" as="geometry">')
                        xml_lines.append('<mxPoint x="0" y="0" as="sourcePoint"/>')
                        xml_lines.append('<mxPoint x="100" y="100" as="targetPoint"/>')
                        xml_lines.append('</mxGeometry>')
                        xml_lines.append('</mxCell>')
                        edge_id += 1
                    except (ValueError, IndexError):
                        continue
    
    # 3. Relations de références croisées (option sets)
    for option_key, references in option_set_references.items():
        if option_key in choice_positions:
            source_choice = choice_positions[option_key]
            source_id = source_choice['id']
            
            for ref in references:
                ref_table = ref['entity']
                ref_column = ref['column']
                if ref_table in table_positions:
                    target_table = table_positions[ref_table]
                    target_table_id = target_table['id']
                    
                    # Trouver l'ID de la colonne cible
                    if ref_table in tables and ref_column in tables[ref_table]:
                        try:
                            col_index = list(tables[ref_table].keys()).index(ref_column)
                            target_attr_id = f"{target_table_id}_attr_{col_index}"
                            
                            # Créer la relation avec style pointillé
                            source_cell = source_id
                            target_cell = target_attr_id
                            xml_lines.append(f'<mxCell id="edge_{edge_id}" value="Référence" style="edgeStyle=orthogonalEdgeStyle;html=1;endArrow=open;endSize=12;dashed=1;strokeColor=#FF9800;strokeWidth=1;fontSize=9;rounded=1;" edge="1" parent="1" source="{source_cell}" target="{target_cell}">')
                            xml_lines.append('<mxGeometry width="100" height="100" relative="1" as="geometry">')
                            xml_lines.append('<mxPoint x="0" y="0" as="sourcePoint"/>')
                            xml_lines.append('<mxPoint x="100" y="100" as="targetPoint"/>')
                            xml_lines.append('</mxGeometry>')
                            xml_lines.append('</mxCell>')
                            edge_id += 1
                        except (ValueError, IndexError):
                            continue
    
    xml_lines.append('</root>')
    xml_lines.append('</mxGraphModel>')
    xml_lines.append('</diagram>')
    xml_lines.append('</mxfile>')
    
    output_file="./data/architecture_complete.drawio"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(xml_lines))
    
    print("✅ Fichier 'architecture_complete.drawio' généré")
    print("Nombre de tables: ", len(tables))
    total_cols = sum(len(cols) for cols in tables.values())
    print("Nombre total de colonnes: ", total_cols)
    
    # Compter les tables de choix générées
    choice_tables_count = len([k for k in option_sets_by_column.keys() if k in option_sets_by_column])
    print("Nombre de tables de choix générées: ", choice_tables_count)


# Afficher les tables avec leurs colonnes et listes de choix
print("=== LISTE DES TABLES AVEC COLONNES ET LISTES DE CHOIX ===")

for table, cols in tables.items():
    print(f"\nTable: {table} ({len(cols)} colonnes)")
    for tech_name, display_name in sorted(cols.items()):
        print(f"  {tech_name} → {display_name}")
        
        # Vérifier s'il y a une liste de choix pour cette colonne
        option_key = f"{table}.{tech_name}"
        if option_key in option_sets_by_column:
            options = option_sets_by_column[option_key]
            print(f"    📋 Liste de choix ({len(options)} valeurs):")
            for key, value in sorted(options.items()):
                print(f"      {key} → {value}")
        
        # Vérifier s'il y a des références
        if option_key in option_set_references:
            refs = option_set_references[option_key]
            print(f"    🔗 Référencé par:")
            for ref in refs:
                print(f"      → {ref['entity']}.{ref['column']}")

# Proposer les exports
print(f"\n=== OPTIONS D'EXPORT ===")
print("1. Mermaid (pour diagramme ER)")
print("2. JSON structuré (complet)")
print("3. CSV des tables et colonnes")
print("4. XML Draw.io")
print("5. HTML (complet avec références)")
print("6. Tous les formats")

choice = input("Quel format voulez-vous générer ? (1/2/3/4/5/6): ").strip()

if choice in ['1', '6']:
    mermaid_content = generate_mermaid_graph(tables)
    output_file = "./data/architecture_mermaid.md"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("# Architecture des données\n\n")
        f.write(mermaid_content)
    print("✅ Fichier 'architecture_mermaid.md' généré")

if choice in ['2', '6']:
    structured_data = generate_structured_json()
    output_file = "./data/architecture_complete.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(structured_data, f, indent=2, ensure_ascii=False)
    print("✅ Fichier 'architecture_complete.json' généré")

if choice in ['3', '6']:
    output_file = "./data/tables_colonnes.csv"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("Table;Nom Technique;Nom Affiché;Avec Liste de Choix;Références\n")
        for table, cols in tables.items():
            for tech_name, display_name in cols.items():
                has_options = "Oui" if f"{table}.{tech_name}" in option_sets_by_column else "Non"
                references = ""
                option_key = f"{table}.{tech_name}"
                if option_key in option_set_references:
                    refs = option_set_references[option_key]
                    references = "|".join([f"{ref['entity']}.{ref['column']}" for ref in refs])
                f.write(f"{table};{tech_name};{display_name};{has_options};{references}\n")
    print("✅ Fichier 'tables_colonnes.csv' généré")

if choice in ['4', '6']:
    generate_drawio_complete(tables, option_sets_by_column, option_set_references)

if choice in ['5', '6']:
    generate_html_export(tables, option_sets_by_column, option_set_references)

print("\n🎉 Extraction terminée !")

=== LISTE DES TABLES AVEC COLONNES ET LISTES DE CHOIX ===

Table: OSI_01_REN_01 (42 colonnes)
  OSI_02 → OSI_01_REN_01_NCDSDSINM_47
    📋 Liste de choix (5 valeurs):
      113790000 → OSI_02_OSNM_05
      113790001 → OSI_02_OSNM_04
      113790002 → OSI_02_OSNM_01
      113790003 → OSI_02_OSNM_02
      113790004 → OSI_02_OSNM_03
    🔗 Référencé par:
      → OSI_01_REN_01.OSI_02
  OSI_03 → OSI_01_REN_01_NCDSDSINM_31
    📋 Liste de choix (9 valeurs):
      1 → OSI_03_OSNM_08
      2 → OSI_03_OSNM_01
      3 → OSI_03_OSNM_02
      4 → OSI_03_OSNM_04
      5 → OSI_03_OSNM_03
      6 → OSI_03_OSNM_05
      7 → OSI_03_OSNM_09
      8 → OSI_03_OSNM_07
      9 → OSI_03_OSNM_06
    🔗 Référencé par:
      → OSI_01_REN_01.OSI_03
      → OSI_01_OSRI_01.OSI_03
  OSI_04 → OSI_01_REN_01_NCDSDSINM_35
    📋 Liste de choix (9 valeurs):
      113790000 → OSI_04_OSNM_08
      113790001 → OSI_04_OSNM_07
      113790002 → OSI_04_OSNM_02
      113790003 → OSI_04_OSNM_03
      113790004 → OSI_04_OSNM_05
     

# La cellule ci dessous anonymise les champs du fichier architecture_complete.drawio

In [4]:
import xml.etree.ElementTree as ET
import random
import string

def anonymiser(chemin_entree, chemin_sortie):
    """Anonymise avec des noms de colonnes cohérents avec les tables"""
    
    # Parse le XML
    tree = ET.parse(chemin_entree)
    root = tree.getroot()
    
    # Dictionnaire pour stocker les noms de tables
    noms_tables = {}
    
    # Première passe : générer les noms de tables
    compteur_tables = 0
    compteur_choix = 0
    
    for cell in root.iter():
        if cell.tag.endswith('mxCell'):
            id_elem = cell.get('id')
            value = cell.get('value', '')
            style = cell.get('style', '')
            
            if id_elem:
                # Tables principales
                if id_elem.startswith('table_') and 'swimlane' in style:
                    compteur_tables += 1
                    # Générer un nom de table cohérent
                    prefixe = random.choice(['Client', 'Commande', 'Produit', 'Utilisateur', 'Facture', 'Stock', 'Fournisseur'])
                    suffixe = random.randint(100, 999)
                    nom_table = f"{prefixe}_{suffixe}"
                    noms_tables[id_elem] = nom_table
                    cell.set('value', nom_table)
                
                # Tables de choix
                elif id_elem.startswith('choice_') and 'swimlane' in style:
                    compteur_choix += 1
                    # Générer un nom de choix cohérent
                    prefixe = random.choice(['Statut', 'Type', 'Categorie', 'Priorite', 'Etat'])
                    suffixe = random.randint(100, 999)
                    nom_choix = f"{prefixe}_{suffixe} [Choix]"
                    cell.set('value', nom_choix)
    
    # Deuxième passe : anonymiser les champs avec référence au nom de table
    for cell in root.iter():
        if cell.tag.endswith('mxCell'):
            id_elem = cell.get('id')
            parent = cell.get('parent')
            value = cell.get('value', '')
            style = cell.get('style', '')
            
            if id_elem and parent:
                # Champs des tables principales
                if '_attr_' in id_elem and parent in noms_tables:
                    nom_table = noms_tables[parent]
                    # Générer un nom de champ lié au nom de table
                    champs_possibles = ['ID', 'Nom', 'Date', 'Montant', 'Statut', 'Description', 'Code', 'Reference']
                    champ_aleatoire = random.choice(champs_possibles)
                    cell.set('value', f"{nom_table}_{champ_aleatoire}")
                
                # Champs des tables de choix
                elif '_option_' in id_elem and parent.startswith('choice_'):
                    cell.set('value', f"Option_{random.randint(1000, 9999)}")
    
    # Écrire le fichier
    tree.write(chemin_sortie, encoding='UTF-8', xml_declaration=True)
    
    print(f"✅ Anonymisation terminée: {chemin_sortie}")
    print(f"Tables principales: {compteur_tables}")
    print(f"Tables de choix: {compteur_choix}")

# Utiliser le script d'anonymisation corrigé
anonymiser('architecture_complete.drawio', 'architecture_anonymise.drawio')


✅ Anonymisation terminée: architecture_anonymise.drawio
Tables principales: 13
Tables de choix: 23
